In [ ]:
%matplotlib inline
#這是jupyter notebook的magic word˙

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

os.environ['TRIDENT_BACKEND'] = 'pytorch'
kaggle_kernal=None
if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'
  


In [ ]:
#為確保安裝最新版 

!pip uninstall tridentx -y
!pip install ../input/trident/tridentx-0.7.4-py3-none-any.whl --upgrade

import re
import pandas
import json
import copy

import numpy as np
#調用trident api
import subprocess
import random
from tqdm import tqdm
import scipy
import time
import glob
import pandas as pd
from shutil import copyfile

In [ ]:
import trident as T
from trident import *

palette = [(255, 192, 0), (0, 192, 255), (128, 0, 255), (192,255,64), (255, 0, 128), (64,255,128)]

In [ ]:
# Read in data files
BASE_DIR = '../input/nfl-health-and-safety-helmet-assignment'

# 標籤與提交範例
train_labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
ss = pd.read_csv(f'{BASE_DIR}/sample_submission.csv')

# 球員追蹤數據
train_player_tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
test_player_tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')

# Baseline helmet detection labels
train_baseline_helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
test_baseline_helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')

# 額外的圖像標註
img_labels = pd.read_csv(f'{BASE_DIR}/image_labels.csv')





img_labels

In [ ]:
extra_images=glob.glob('../input/nfl-health-and-safety-helmet-assignment/images/*.jpg')
print(len(extra_images))

img_labels_frequency=img_labels['label'].value_counts()
print(img_labels_frequency)

label2index = {
    'Helmet': 0,
    'Helmet-Blurred': 1,
    'Helmet-Difficult': 2,
    'Helmet-Sideline': 3,
    'Helmet-Partial': 4
}

folder,filename,ext=split_path(extra_images[0])
sample_data=img_labels[img_labels['image']==filename+ext]
sample_data


In [ ]:
img=image2array(extra_images[0])
pillow_img=array2image(img)

for index, item in sample_data.iterrows():
    box=[int(item['left']),int(item['top']),int(item['left'])+int(item['width']),int(item['top'])+int(item['height'])] 
    pillow_img =plot_bbox(box, pillow_img, palette[label2index[item['label']]],item['label'], line_thickness=2)

display.display(pillow_img)

In [ ]:
#額外圖片的數據
bboxes_dataset_dict=OrderedDict()
for img_path in tqdm(extra_images):
    folder,filename,ext=split_path(img_path)
    sub_data=img_labels[img_labels['image']==filename+ext]
    bboxes_dataset_dict[filename+ext]=[]

    for index, item in sub_data.iterrows():
        box=[int(item['left']),int(item['top']),int(item['left'])+int(item['width']),int(item['top'])+int(item['height'])]
        bboxes_dataset_dict[filename+ext].append(box)
    bboxes_dataset_dict[filename+ext]=np.array(bboxes_dataset_dict[filename+ext])
        
print('bboxes_dataset_dict:',len(bboxes_dataset_dict))
pickle_it('./bboxes_dataset_dict.pkl',bboxes_dataset_dict)

In [ ]:
from matplotlib import animation, rc
from IPython.display import Video, display

def display_video(video_path, ratio=0.5):
    return Video(f"{video_path}",
                  embed=True,
                  height = int(720 * ratio),
                  width = int(1280 * ratio))
train_videos=glob.glob('../input/nfl-health-and-safety-helmet-assignment/train/*.mp4')

display_video( train_videos[0])

In [ ]:
train_baseline_helmets["video"] = train_baseline_helmets["video_frame"].str.split("_").str[:3].str.join("_")
train_labels["video"] = train_labels["video_frame"].str.split("_").str[:3].str.join("_")

train_baseline_helmets["frame"] = train_baseline_helmets["video_frame"].str.split("_").str[-1].astype("int")
train_labels["frame"] = train_labels["video_frame"].str.split("_").str[-1].astype("int")

In [ ]:
import IPython
VIDEO_CODEC = "mp4v"

video_bboxes_dataset_dict=OrderedDict()

make_dir_if_need('./videos')
for video_path in tqdm(train_videos):
    folder,filename,ext=split_path(video_path)
    make_dir_if_need('./videos/{0}'.format(filename))
    video_name = os.path.basename(video_path).replace(".mp4", "")
    videocap = cv2.VideoCapture(video_path)
    fps = videocap.get(cv2.CAP_PROP_FPS)
    width = int(videocap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(videocap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame = 0
    while True:
        it_worked, img = videocap.read()
        if not it_worked:
            break
        # We need to add 1 to the frame count to match the label frame index
        # that starts at 1
        frame += 1
        if random.random()<=0.035:

        # Let's add a frame index to the video so we can track where we are
      

#         # Now, add the boxes
#         boxes = train_baseline_helmets.query("video == @video_name and frame == @frame")
#         if len(boxes) == 0:
#             print("Boxes incorrect")
#             return 
        
  
#         bboxes_dataset_dict[filename+ext]=[]
#         for box in boxes.itertuples(index=False):
#             bboxes_dataset_dict[filename+ext].append([box.left, box.top,box.left+box.width, box.top+box.height])
   
    

            boxes = train_labels.query("video == @video_name and frame == @frame")
            if len(boxes) == 0:
                pass
            else:
                print('./videos/{0}/{1}_{2}.png'.format(filename,filename,frame),'boxes:',len(boxes))
                array2image(img).save('./videos/{0}/{1}_{2}.png'.format(filename,filename,frame))
                video_bboxes_dataset_dict['{0}_{1}.png'.format(filename,frame)]=[]
                for box in boxes.itertuples(index=False):
                    video_bboxes_dataset_dict['{0}_{1}.png'.format(filename,frame)].append([box.left, box.top,box.left+box.width, box.top+box.height])
                video_bboxes_dataset_dict['{0}_{1}.png'.format(filename,frame)]=np.array(video_bboxes_dataset_dict['{0}_{1}.png'.format(filename,frame)])
IPython.display.clear_output(wait=True)
print('video_bboxes_dataset_dict:',len(video_bboxes_dataset_dict))
pickle_it('./video_bboxes_dataset_dict.pkl',video_bboxes_dataset_dict)  
    

In [ ]:
from IPython import display
k=video_bboxes_dataset_dict.key_list[0]

print(glob.glob('./videos/*/'+k)[0])
img=image2array(glob.glob('./videos/*/'+k)[0])
pillow_img=array2image(img)

for box in video_bboxes_dataset_dict[k]:
    pillow_img =plot_bbox(box, pillow_img, palette[0] ,'Helmet', line_thickness=2)

display.display(pillow_img)

In [ ]:
def video_with_baseline_boxes(video_path, baseline_boxes, gt_labels, verbose=True):
    """
    Annotates a video with both the baseline model boxes and ground truth boxes.
    Baseline model prediction confidence is also displayed.
    """
    VIDEO_CODEC = "mp4v"
    HELMET_COLOR = (255, 0, 0) # Blue 
    BASELINE_COLOR = (0, 255, 0) # Green
    IMPACT_COLOR = (0, 0, 255) # Red
    video_name = os.path.basename(video_path).replace(".mp4", "")
    
    if verbose:
        print(f"Running for {video_name}")
    baseline_boxes = baseline_boxes.copy()
    gt_labels = gt_labels.copy()

    baseline_boxes["video"] = baseline_boxes["video_frame"].str.split("_").str[:3].str.join("_")
    gt_labels["video"] = gt_labels["video_frame"].str.split("_").str[:3].str.join("_")

    baseline_boxes["frame"] = baseline_boxes["video_frame"].str.split("_").str[-1].astype("int")
    gt_labels["frame"] = gt_labels["video_frame"].str.split("_").str[-1].astype("int")

    videocap = cv2.VideoCapture(video_path)
    fps = videocap.get(cv2.CAP_PROP_FPS)
    width = int(videocap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(videocap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_path = f"labelled_{video_name}.mp4"
    tmp_output_path = f"tmp_" + output_path
    output_video = cv2.VideoWriter(
                        tmp_output_path,
                        cv2.VideoWriter_fourcc(*VIDEO_CODEC),
                        fps,
                        (width, height)
                    )
    frame = 0
    while True:
        it_worked, img = videocap.read()
        if not it_worked:
            break
        # We need to add 1 to the frame count to match the label frame index
        # that starts at 1
        frame += 1

        # Let's add a frame index to the video so we can track where we are
        img_name = f"{video_name}_frame{frame}"
        cv2.putText(img, img_name, (0, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1.0, HELMET_COLOR, thickness=2)

        # Now, add the boxes
        boxes = baseline_boxes.query("video == @video_name and frame == @frame")
        if len(boxes) == 0:
            print("Boxes incorrect")
            return 
        for box in boxes.itertuples(index=False):
            cv2.rectangle(img, (box.left, box.top), (box.left+box.width, box.top+box.height),
                          BASELINE_COLOR, thickness=1)
            cv2.putText(img, f"{box.conf:.2f}", (box.left, max(0, box.top-5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, BASELINE_COLOR, 1)

        boxes = gt_labels.query("video == @video_name and frame == @frame")
        if len(boxes) == 0:
            print("Boxes incorrect")
            return 
        for box in boxes.itertuples(index=False):
            # Filter for definitive head impacts and turn red
            if box.isDefinitiveImpact == True:
                color, thickness = IMPACT_COLOR, 3
            else:
                color, thickness = HELMET_COLOR, 1
            cv2.rectangle(img, (box.left, box.top), (box.left+box.width, box.top+box.height),
                          color, thickness=thickness)
            cv2.putText(img, box.label, (box.left+1, max(0, box.top-20)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness=1)

        output_video.write(img)
    output_video.release()
    # Not all browsers support the codec, we will re-load the file at tmp_output_path
    # and convert to a codec that is more readable using ffmpeg
    if os.path.exists(output_path):
        os.remove(output_path)
    subprocess.run([
        "ffmpeg",
        "-i",
        tmp_output_path,
        "-crf",
        "18",
        "-preset",
        "veryfast",
        "-vcodec",
        "libx264",
        output_path
    ])
    os.remove(tmp_output_path)

    return output_path

In [ ]:
from matplotlib import animation, rc
from IPython.display import Video, display
output_video = video_with_baseline_boxes(train_videos[0],train_baseline_helmets, train_labels)

frac = 1.0 # scaling factor for display
display(Video(data=output_video,
              embed=True,
              height=int(720*frac),
              width=int(1280*frac)))

In [ ]:
# def create_animation(num_frames=10, interval=1000):
#     fig = plt.figure(figsize=(12, 6))
#     plt.axis('off')
    
#     image_bboxed = get_random_bboxed_image()
#     image = plt.imshow(image_bboxed)
    
#     def animate_func(i):
#         image_bboxed = get_random_bboxed_image()
#         image.set_array(image_bboxed)
#         return [image]
    
#     return animation.FuncAnimation(fig,
#                                    animate_func,
#                                    frames=num_frames,
#                                    interval=interval # in ms
#                                   )
# rc('animation', html='jshtml')

In [ ]:
from trident.models import ssd,rfbnet

#pretrainedmodel=rfbnet.RfbNet(pretrianed=True)
#pretrainedmodel.summary()

rfbmodel=rfbnet.RfbNet(pretrianed=True, num_classes=2, num_regressors=4)
print(rfbmodel.model.priors)
rfbmodel.summary()

In [ ]:
np.set_printoptions(precision=3)

def point_in_box(point,box):
    if ndim(box)+1==ndim(point):
        box=np.expand_dims(box,1)
    if ndim(point)==ndim(box)==1:
        x,y=point
        x1,y1,x2,y2=box
        if x1<x<x2 and y1<y<y2:
            return True
        else:
            return False
    elif  ndim(point)==ndim(box)==2:
        x=point[:,0]
        y=point[:,1]
        x1=box[:,0]
        y1=box[:,1]
        x2=box[:,2]
        y2=box[:,3]
        return np.greater_equal(x,x1)*np.greater_equal(x2,x)*np.greater_equal(y,y1)*np.greater_equal(y2,y)
    elif  ndim(point)==ndim(box)==3:
        x=point[:,:,0]
        y=point[:,:,1]
        x1=box[:,:,0]
        y1=box[:,:,1]
        x2=box[:,:,2]
        y2=box[:,:,3]
        return np.greater_equal(x,x1)*np.greater_equal(x2,x)*np.greater_equal(y,y1)*np.greater_equal(y2,y)
   

In [ ]:

def xywh2xyxy(locations):
    return np.concatenate([locations[..., :2] - locations[..., 2:] / 2.0, locations[..., :2] + locations[..., 2:] / 2.0],
                     axis=1)

class SsdBboxDataset(BboxDataset):
    def __init__(self, boxes=None, image_size=(480, 640), priors=None, center_variance=0.1, size_variance=0.2,
                 gt_overlap_tolerance=0.5, object_type=ObjectType.absolute_bbox, class_names=None,
                 symbol='bbox', name=''):
        super().__init__(boxes=boxes, image_size=image_size, object_type=object_type, class_names=class_names,
                         symbol=symbol, name=name)
        self._element_spec = TensorSpec(shape=TensorShape([None, 21]), name=self.symbol, object_type=self.object_type, is_spatial=True)
        self.priors = priors
        self.image_size = image_size
        self.center_variance = center_variance
        self.size_variance = size_variance
        self.label_transform_funcs = []
        self.gt_overlap_tolerance = gt_overlap_tolerance
        self.bbox_post_transform_funcs = []
        print('priors',self.priors.shape,self.priors )

    def __getitem__(self, index: int):

        # 取出xyxy定義boxes或是[]
        # 為了避免[]清單與np.ndarray計算差異，因此補上np.array([[-1,-1,-1,-1,-1]])替代
        if self.items[index] is None or self.items[index] == []:
            return np.array([-1] * 21)
        else:
            
            return self.items[index].astype(np.float32)

    def area_of(self, left_top, right_bottom):
        """Compute the areas of rectangles given two corners.

        Args:
            left_top (N, 2): left top corner.
            right_bottom (N, 2): right bottom corner.

        Returns:
            area (N): return the area.
        """
        hw = np.clip(right_bottom - left_top, 0.0, None)
        return hw[..., 0] * hw[..., 1]

    def iou_of(self, boxes0, boxes1, eps=1e-5):
        """Return intersection-over-union (Jaccard index) of boxes.

        Args:
            boxes0 (N, 4): ground truth boxes.
            boxes1 (N or 1, 4): predicted boxes.
            eps: a small number to avoid 0 as denominator.
        Returns:
            iou (N): IoU values.
        """
        overlap_left_top = np.maximum(boxes0[..., :2], boxes1[..., :2])
        overlap_right_bottom = np.minimum(boxes0[..., 2:], boxes1[..., 2:])

        overlap_area = self.area_of(overlap_left_top, overlap_right_bottom)
        area0 = self.area_of(boxes0[..., :2], boxes0[..., 2:])
        area1 = self.area_of(boxes1[..., :2], boxes1[..., 2:])
        return overlap_area / (area0 + area1 - overlap_area + eps)

    def convert_boxes_to_locations(self, center_form_boxes, center_form_priors):
        if len(center_form_priors.shape) + 1 == len(center_form_boxes.shape):
            center_form_priors = np.expand_dims(center_form_priors, 0)
        return np.concatenate([(center_form_boxes[..., :2] - center_form_priors[..., :2]) / center_form_priors[...,
                                                                                            2:] / self.center_variance,
                               np.log(np.clip(center_form_boxes[..., 2:] / center_form_priors[..., 2:], 1e-8, np.inf)) / self.size_variance],
                              axis=len(center_form_boxes.shape) - 1)

    def assign_priors(self, gt_boxes, gt_labels,center_form_priors, iou_threshold):

        corner_form_priors = xywh2xyxy(center_form_priors)
        #print('corner_form_priors',corner_form_priors.shape,corner_form_priors)
        ious = self.iou_of(np.expand_dims(gt_boxes, 0), np.expand_dims(corner_form_priors, 1))
    
        # size: num_priors
        best_target_per_prior, best_target_per_prior_index = np.max(ious, axis=1), np.argmax(ious, axis=1)
    
        best_prior_per_target, best_prior_per_target_index = np.max(ious, axis=0), np.argmax(ious, axis=0)
    
        
        for target_index, prior_index in enumerate(best_prior_per_target_index):
            best_target_per_prior_index[prior_index] = target_index
        
        #2.0 is used to make sure every target has a prior assigned
        best_prior_per_target_index_list = best_prior_per_target_index.tolist()
        for i in range(best_target_per_prior.shape[0]):
            if i in best_prior_per_target_index_list:
                best_target_per_prior[i] = 2


       
        labels= gt_labels[best_target_per_prior_index]
        labels[best_target_per_prior<2]=0
    
        boxes = gt_boxes[best_target_per_prior_index]


        return boxes, labels

    def data_transform(self, data):
        if data is None or len(data) == 0:
            return np.zeros((self.priors.shape[0], 21)).astype(np.float32)
        elif isinstance(data, np.ndarray):
            height, width = self.image_size

            data[:, 0] = np.clip(data[:, 0], 0, width)
            data[:, 2] = np.clip(data[:, 2], 0, width)
            data[:, 1] = np.clip(data[:, 1], 0, height)
            data[:, 3] = np.clip(data[:, 3], 0, height)
            
            
            small_box_mask1=np.round(data[:, 0])==np.round(data[:, 2],0)
            data[:, 2:3][small_box_mask1,:]+=1
            small_box_mask2=np.round(data[:, 1])==np.round(data[:, 3],0)
            data[:, 3:4][small_box_mask2,:]+=1
            
            
            box_w = np.clip(np.expand_dims(data[:, 2] - data[:, 0], -1),1,640)
            box_h = np.clip(np.expand_dims(data[:, 3] - data[:, 1], -1),1,480)
            box_left = np.expand_dims(data[:, 0].copy(), -1)
            box_top = np.expand_dims(data[:, 1].copy(), -1)
            # print('box_w',box_w.shape)
            # print('box_left',box_left.shape)

            gt_box = data[:, :4]
            gt_label = data[:, 4:5]
        
            gt_box[:, 0] = gt_box[:, 0] / float(width)
            gt_box[:, 2] = gt_box[:, 2] / float(width)
            gt_box[:, 1] = gt_box[:, 1] / float(height)
            gt_box[:, 3] = gt_box[:, 3] / float(height)

        

            if gt_box is not None and len(gt_box) > 0:
                truths = to_tensor(gt_box).float()
                labels = to_tensor(gt_label).long()
      
                
                boxes, confidences = self.assign_priors(gt_box, gt_label,  to_numpy(self.priors.copy()), 0.5)
                boxes = xyxy2xywh(boxes)
                locations = self.convert_boxes_to_locations(boxes, to_numpy(self.priors.copy()))

                locations = np.concatenate([to_numpy(locations).astype(np.float32),to_numpy(confidences).astype(np.float32)], axis=-1)
                return to_numpy(locations).astype(np.float32)

            num_priors = self.priors.shape[0]
            return np.zeros((num_priors, 5)).astype(np.float32)


    def bbox_transform(self, bbox):
        return self.data_transform(bbox)
